In [1]:
import numpy as np
import cv2 as cv
import os

In [2]:
# img=cv.imread("person.jpg")
# cv.imshow("MY_IMAGE",img)

In [3]:
# gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# cv.imshow("Gray_IMAGE",gray)


In [4]:
# ## Importing of haar cascade
# hc=cv.CascadeClassifier("haar_face.xml")

In [5]:
# face_react=hc.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3)

In [6]:
# #No of faces
# len(face_react)

In [7]:
# #now drawing box around face
# for (x,y,w,h) in face_react:
#     cv.rectangle(img,(x,y),(x+w,y+h),thickness=3,color=(0,255,0))

# cv.imshow("Detected_face",img)

In [8]:
# import cv2 as cv

# # Load the pre-trained face detection model (Haar Cascade)
# hc = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

# # Function to detect and display faces in an image
# def detect_faces(img):
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     face_rects = hc.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#     for (x, y, w, h) in face_rects:
#         cv.rectangle(img, (x, y), (x + w, y + h), color=(100, 0, 0), thickness=3)

#     return img

# # Capture video from the default camera (index 0)
# cap = cv.VideoCapture(0)

# if not cap.isOpened():
#     print("Error: Could not open video source.")
#     exit()

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Error: Cannot read frame.")
#         break

#     # Detect faces in the frame
#     frame_with_faces = detect_faces(frame)
#     face=cv.resize(frame_with_faces,   (1080,720))

#     # Display the resulting frame
#     cv.imshow("Face Detection", faces)

#     # Exit the loop when 'x' key is pressed
#     if cv.waitKey(1) & 0xFF == ord('x'):
#         break

# # Release the capture and close windows
# cap.release()
# cv.destroyAllWindows()


In [9]:
feeling={}
dir=r"C:\Users\HP\OneDrive\Desktop\semi project 5\train"
for emotion in os.listdir(r"C:\Users\HP\OneDrive\Desktop\semi project 5\train"):
    feeling[emotion]=len(os.listdir(os.path.join(dir,emotion)))

In [10]:
feeling

{'angry': 3995,
 'disgusted': 436,
 'fearful': 4097,
 'happy': 7215,
 'neutral': 4965,
 'sad': 4830,
 'surprised': 3171}

In [11]:
feel=list(feeling.keys())
features=[]
labels=[]

In [12]:
feel

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [13]:
haar_cascade=cv.CascadeClassifier('haar_face.xml')

In [14]:
# creating of data
def create_train():
    for f in feel:
        path=os.path.join(dir,f)
        label=feel.index(f)
        for img in os.listdir(path):
            img_path=os.path.join(path,img)
            img_array=cv.imread(img_path)
            gray=cv.cvtColor(img_array,cv.COLOR_BGR2GRAY)
            face_react=haar_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3)
            for (x,y,w,h) in face_react:
                faces_roi=gray[y:y+h,x:x+w]
                #resize it so that it can be converted to np array
                faces_roi_resized = cv.resize(faces_roi, (100,100))

                features.append(faces_roi_resized)
                labels.append(label)
                

In [15]:
create_train()

In [16]:
features

[array([[47, 48, 52, ..., 55, 52, 51],
        [49, 49, 54, ..., 55, 52, 51],
        [58, 60, 67, ..., 56, 52, 52],
        ...,
        [75, 75, 77, ..., 51, 51, 51],
        [73, 73, 75, ..., 52, 49, 48],
        [73, 73, 75, ..., 52, 49, 48]], dtype=uint8),
 array([[50, 49, 46, ..., 67, 68, 68],
        [49, 49, 46, ..., 66, 67, 67],
        [47, 46, 45, ..., 62, 62, 62],
        ...,
        [35, 34, 32, ..., 15, 13, 13],
        [42, 41, 34, ..., 19, 21, 21],
        [43, 42, 34, ..., 20, 22, 22]], dtype=uint8),
 array([[ 84,  84,  87, ..., 123, 123, 123],
        [ 85,  85,  88, ..., 125, 124, 123],
        [ 88,  89,  92, ..., 131, 126, 126],
        ...,
        [ 82,  83,  86, ..., 115, 125, 128],
        [ 81,  82,  86, ..., 107, 119, 121],
        [ 81,  82,  86, ..., 105, 117, 120]], dtype=uint8),
 array([[ 48,  45,  38, ...,  41,  42,  43],
        [ 50,  47,  38, ...,  40,  42,  43],
        [ 56,  52,  38, ...,  39,  42,  43],
        ...,
        [ 23,  23,  21, ..., 1

In [17]:
#model training
# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

In [18]:
len(features)

10598

In [19]:
# Create and train the face recognizer
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.train(features, labels)

In [20]:
#Save all
np.save('labels.npy',labels)
np.save('features.npy',features)
face_recognizer.save('face_trained.yml')

In [21]:
cap=cv.VideoCapture(0)

while True:
    ret,frame=cap.read()
    if(ret==False):
        print("Camera not found")
        break
    else:
        gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        #Detect the image in frame
        faces_react=haar_cascade.detectMultiScale(gray,1.1,4)
        for (x,y,w,h) in faces_react:
            faces_roi=gray[y:y+h,w:w+h]
            faces_resized=cv.resize(faces_roi,(100,100))
            label,confidence=face_recognizer.predict(faces_resized)
            cv.rectangle(frame,(x,y),(x+w,y+h),color=(0,255,0),thickness=3)
            cv.putText(frame,str(feel[label]),(x+w//2,y+h+30),cv.FONT_HERSHEY_COMPLEX,1.0,(0,255,0),thickness=2)
            resized=cv.resize(frame,(720,480))
            cv.imshow("LIVE",resized)

    if(cv.waitKey(1) & 0xFF==ord('x')):
        break

cap.release()
cv.destroyAllWindows()